# Trabajo Práctico 2: Pruebas de validez

Las pruebas de validez nos permiten probar que un cómputo se ha realizado correctamente en un tiempo bastante menor que la verificación trivial, es decir, que el cómputo sea corrido de manera independiente por el verificador. En este trabajo práctico, nos enfocaremos en el sistema de pruebas STARKs en forma simplificada. Para el trabajo práctico, se podrá hacer uso de las herramientas provistas en https://github.com/starkware-industries/stark101 (python) o https://github.com/lambdaclass/STARK101-rs/tree/main (rust).

## Objetivo

El objetivo consiste en probar la validez de un programa que calcula 2^{8^20} módulo p, utilizando diferentes estrategias y viendo el perfil de costos (tanto en términos de longitud de la prueba como de tiempo de generación). Las estrategias propuestas son:

1. Considerar $a_0 = 2$ y $a_{n+1}=a_n^8$.
2. Considerar $a_0 = 2$ y $a_{n+1}=a_n^2$.
3. Considerar $a_0 = 2$ y $a_{2n+1}=a_{2n}^2$ y $a_{2n}=a_{2n-1}^4$.
4. Usando 2 columnas

Tener en cuenta que, dado el grado de las restricciones, se requiere usar un factor de expansión (blowup) por lo menos tan grande como el grado de las restricciones.

## Propuesta 1: $a_0 = 2$ y $a_{n+1}=a_n^8$

### Polinomio de la traza
Primero vamos a generar la traza. La misma es una lista de 21 elementos, siendo el primero de ellos $a_0 = 2$ y siendo $a_n=a_{n-1}^8$ para $1 \le n \le 20$. De esta manera, el último elemento será $a_{20}=a_{19}^8=2^{8^{20}}$.

In [1]:
from utils.field import FieldElement

a0 = FieldElement(2)
a = [a0]
for i in range(20):
    a.append(a[-1]**8)

In [2]:
#Algunos chequeos de la traza
assert len(a) == 21
assert a[0] == FieldElement(2)
assert a[-1] == FieldElement(2)**(8**20)
print("Traza creada correctamente")

Traza creada correctamente


Ahora hallamos un generador $g$ que nos permita generar un grupo de tamaño 21. 

Sabiendo que $g^k$ genera un grupo de tamaño $\frac {|\mathbb{F}^\times|}{k}$ cuando $k$ divide a $|\mathbb{F}^\times|$, debemos hallar $k$ tal que:

$
21 = \frac {|\mathbb{F}^\times|}{k} \\
21 = \frac{3221225472}{k} \\
k = \frac{3221225472}{21} = 153391689.143
$

Como vemos, $k$ nos queda como un número decimal. Entonces, debemos buscar un número potencia de 2 más próximo a 21 (y más grande) tal que divida a 3221225472, es decir, 32, por eso lo fijamos como el tamaño de nuestra traza. Ahora sí, calculamos $k$

$
32 = \frac {|\mathbb{F}^\times|}{k} \\
32 = \frac{3221225472}{k} \\
k = \frac{3221225472}{32} = 100663296
$

In [3]:
import math

def get_group(generator, trace_len):
    size = 2 ** math.ceil(math.log2(trace_len))
    k = 3*(2**30)/size
    g = generator ** (k)
    G = [] 
    for i in range(size):
        G.append(g**i)
    
    return g,G

In [4]:
generator = FieldElement.generator()
g,G = get_group(generator,len(a))

In [5]:
#Algunos chequeos del grupo generado
assert g.is_order(32), print("El grupo no tiene orden 32")
assert G[-1]*g == FieldElement(1), print("No es ciclico")
print("Grupo generado correctamente")

Grupo generado correctamente


Pr último, para obtener el polinomio de la traza interpolamos los 21 puntos $(x_i,y_i)$ tal que:
- $x_i$: es el elemento $i$ del grupo $G$ generado.
- $y_i$: es el elemento $i$ de la traza

In [6]:
from utils.polynomial import interpolate_poly
f = interpolate_poly(G[:21],a)

### Expansión
Ahora realizamos la expansión, donde evaluamos nuestro polinomio sobre un dominio más grande. Para eso, creamos un grupo nuevo $H$ el cual será 8 veces más grande que $G$ y posteriormente, obtenemos el dominio de evaluación tal que

$$eval\_domain = \{w\cdot h^i | 0 \leq i <256  \}$$

In [10]:
from utils.merkle import MerkleTree
from utils.channel import Channel

def expand_domain(length):
    w = FieldElement.generator()
    h,H =get_group(w, length)
    eval_domain = [w*(h**i) for i in range(len(H))]
    return eval_domain

#Obtenemos dominio de evaluacion
eval_domain = expand_domain(len(G)*8)

#Evaluamos en el polinomio de la traza
f_eval = [f(x) for x in eval_domain]

#Armamos el Merkle tree con las evaluaciones
f_merkle = MerkleTree(f_eval)

#Instanciamos un canal y enviamos la raíz del árbol
channel = Channel()
channel.send(f_merkle.root)

### Restricciones

Tenemos tres restricciones a tener en cuenta:
1. El primer elemento de la traza debe ser igual a $2$. Esto es $a[0] = 2$.
2. El último elemento de la traza debe ser igual a $2^{8^{20}}$. Esto es $a[-1] = 2^{8^{20}} mod(p) = 1610563584$
3. Se debe cumplir que $a[i+1] = a[i]^8$ para todo $0 \leq i \leq 19$

In [26]:
from utils.polynomial import X, prod

#Primera Restricción
numer0 = f - 2
denom0 = X - 1

p0 = numer0 / denom0

#Segunda Restricción
numer1 = f - 1610563584
denom1 = X - g**20

p1 = numer1 / denom1

#Tercera Restricción
numer2 = f(g*X) - f**8



## Propuesta 2: $a_0 = 2$ y $a_{n+1}=a_n^2$

## Propuesta 3: $a_0 = 2$ y $a_{2n+1}=a_{2n}^2$ y $a_{2n}=a_{2n-1}^4$

## Propuesta 4: Usar dos columnas